In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# data_dir = 'gdrive/MyDrive/kaggle/ventilator-pressure-prediction/data'
# SAVE_MODEL_PATH = 'gdrive/MyDrive/kaggle/ventilator-pressure-prediction/code/models/model_class28'


data_dir = '../input/ventilator-pressure-prediction'
SAVE_MODEL_PATH = '/kaggle/working/'

In [ ]:
# !pip install pandas --upgrade
!pip install tez
#!pip install Time2Vec-PyTorch

In [ ]:
# !pip install transformers > /dev/null
# !pip install git+https://github.com/rashmibanthia/tez.git@b81953fa2e5ae2f4ea8d560de35c3a857c5e59e6 > /dev/null

In [ ]:
import os
import torch
import tez

import random
import gc 

import pandas as pd
import torch.nn as nn
import numpy as np

from sklearn import metrics
from tez.callbacks import EarlyStopping
from tqdm import tqdm

import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup
from transformers import get_cosine_schedule_with_warmup
from tqdm.notebook import tqdm as tqdm
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR,ReduceLROnPlateau,CosineAnnealingWarmRestarts
from sklearn.preprocessing import RobustScaler

In [ ]:
def seed_everything(seed: int) -> None:
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, w):
        if y is None:
            y = np.zeros(len(X), dtype=np.float32)

        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.w = w.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i], self.w[i]

In [ ]:
# change fold here
class args:
    fold = 4
    learning_rate = 2e-3  
    batch_size = 1024
    epochs = 300
    accumulation_steps = 1
    output_folder = SAVE_MODEL_PATH
    fp16 = False

    device = 'cuda'
    input_dim = 13 #8 #5
    dense_dim = 512
    lstm_dim = 512
    logit_dim = 512

    EMBED_SIZE = 64
    HIDDEN_SIZE = 256
    
#     ALL_FEATURES = ['u_in', 'u_out', 'time_step', 'u_in_cumsum', 'u_in_cummean', 'area', 'cross', 'cross2', 'R_cate', 'C_cate', 'RC_sum', 'RC_dot',
#  'u_in_lag_1','u_in_lag_2','u_in_lag_3','u_in_lag_4', 'u_in_time1', 'u_in_time2' , 'u_in_time3', 'u_in_time4',
# 'u_out_lag_1','u_out_lag_2','u_out_lag_3','u_out_lag_4']

    ALL_FEATURES = ['time_step', 'u_in', 'u_out', 
       'area', 'u_in_cumsum', 'u_in_lag1', 'u_out_lag1', 'u_in_lag_back1',
       'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2', 'u_in_lag_back2',
       'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3', 'u_in_lag_back3',
       'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4', 'u_in_lag_back4',
       'u_out_lag_back4', 'breath_id__u_in__max', 'breath_id__u_out__max',
       'u_in_diff1', 'u_out_diff1', 'u_in_diff2', 'u_out_diff2',
       'breath_id__u_in__diffmax', 'breath_id__u_in__diffmean', 'u_in_diff3',
       'u_out_diff3', 'u_in_diff4', 'u_out_diff4', 'cross', 'cross2', 'R_20',
       'R_5', 'R_50', 'C_10', 'C_20', 'C_50', 'R__C_20__10', 'R__C_20__20',
       'R__C_20__50', 'R__C_50__10', 'R__C_50__20', 'R__C_50__50',
       'R__C_5__10', 'R__C_5__20', 'R__C_5__50', 
       'u_in_1st_derivative', 'expand_mean_1sr_der', 'u_in_1st_der_mean10',
       'time_diff', 'power', 'power_cumsum', 'ewm_u_in_mean', 'ewm_u_in_std',
       'ewm_u_in_corr', 'delta_u_in', 'delta_u_in_exp',
       'delta_rolling_10_mean', 'delta_rolling_10_max', 'work', 'work_roll_10',
       'work_roll_15', 'u_in_rol_q0.1', 'u_in_rol_q0.25', 'u_in_rol_q0.5',
       'u_in_rol_q0.75', 'u_in_rol_q0.9']

    
print(len(args.ALL_FEATURES))

In [ ]:
# !cp ../input/ventilator-train-classification-ce-pytorch-optim/*.csv /kaggle/working/
# !cp ../input/ventilator-train-classification-ce-pytorch-optim/model_f0_r1.bin /kaggle/working/model_f0_split6.bin

In [ ]:
import pandas as pd
from sklearn import model_selection
#df=pd.read_csv('../input/ventilator-train-classification-ce-pytorch-optim/train_folds.csv')
''' 
df=pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df["kfold"] = -1
y = df.pressure.values

kf = model_selection.GroupKFold(n_splits=7)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y, groups=df.breath_id.values)):
    df.loc[v_, "kfold"] = f

df.to_csv("train_folds.csv", index=False)
''' 

df=pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df["kfold"] = -1
y = df.pressure.values
kf = model_selection.GroupKFold(n_splits=7)
for f, (t_, v_) in enumerate(kf.split(X=df, y=y, groups=df.breath_id.values)):
    df.loc[v_, "kfold"] = f
df.to_csv("train_folds7.csv", index=False)

In [ ]:
 #df_uout_0=df[df.u_out==0].reset_index(drop=True)

In [ ]:
#df['cycle_last_ts']=df.groupby(['breath_id','u_out'])['time_step'].transform('max') 
#df['cycle_last_ts_diff']=df['cycle_last_ts']-df['time_step']

In [ ]:
#df[(df.breath_id==1) & (df.u_out==0)]
df.iloc[:,2:5] 

In [ ]:
seed_everything(42)

# ====================================================
# Data Loading
# ====================================================
train_df = df 
#test_df = pd.read_csv(f'{data_dir}/test.csv')
sub = pd.read_csv(f'{data_dir}/sample_submission.csv')

for c in ['u_in']:
    train_df[c] = np.log1p(train_df[c])
    #test_df[c] = np.log1p(test_df[c])



In [ ]:
oof = np.zeros(len(train_df)).astype('float32')
test_preds_lst = []

target_dic = {v:i for i, v in enumerate(sorted(train_df['pressure'].astype('float32').unique().tolist()))}
target_dic_inv = {v: k for k, v in target_dic.items()}
gc.collect()

In [ ]:
#train_df=df
#target_dict_inv_array=np.zeros(len(target_dic_inv))
target_dict_inv_array=np.array(list(target_dic_inv.values())).astype('float32')
target_dict_inv_array.shape
target_dict_inv_array=torch.tensor(target_dict_inv_array) 

del target_dic_inv

In [ ]:
#target_dict_inv_array[27] ,target_dic 

In [ ]:
train_df['tc']=train_df['R']*(train_df['C']/1000)
#test_df['tc']=test_df['R']*(test_df['C']/1000)

train_df['id']=train_df.id.astype('int32')
train_df['R']=train_df.R.astype('int32')
train_df['C']=train_df.C.astype('int32')
train_df['u_out']=train_df.u_out.astype('int32')
train_df['time_step']=train_df.time_step.astype('float32')
train_df['u_in']=train_df.u_in.astype('float32')
train_df['pressure']=train_df.pressure.astype('float32')
train_df['kfold']=train_df.kfold.astype('int8')

In [ ]:
#import cudf

In [ ]:
#train_df=cudf.DataFrame().from_pandas(train_df)

In [ ]:
gc.collect()

#train_df.groupby(train_df['breath_id'])['u_in'].rolling(window=10, min_periods=1)#.fillna(0)#.quantile(0.9)#.reset_index(level=0,drop=True)     
#train_df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)    

#train_df.groupby('breath_id')['u_in'].expanding(2).mean().reset_index(level=0,drop=True)
#train_df.groupby('breath_id')['u_in'].rolling(window=10, min_periods=1).mean().reset_index( drop=True) 
embedding_df=df.iloc[:,2:5]   

In [ ]:
def add_feature(df):
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()
    
    #df = df.drop(['time_delta','delta'], axis=1)
    df = df.drop(['delta'], axis=1)
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df["u_out_sum"] = df.groupby("breath_id")["u_out"].transform("sum").astype('int32')
    df['cycle_last_ts']=df.groupby(['breath_id','u_out'])['time_step'].transform('max') 
    df['cycle_last_ts_diff']=df['cycle_last_ts']-df['time_step']
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1).fillna(0).astype('float32')
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1).fillna(0).astype('float32')
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2).fillna(0).astype('float32')
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2).fillna(0).astype('float32')
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    #df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3).fillna(0).astype('float32')
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    #df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3).fillna(0).astype('float32')
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    #df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4).fillna(0).astype('float32')
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    #df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4).fillna(0).astype('float32')
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    #df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = (df['u_out'] - df['u_out_lag1']).fillna(0).astype('float32')
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = (df['u_out'] - df['u_out_lag2']).fillna(0).astype('float32')
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    #df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    #df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    #df['u_out_diff3'] = (df['u_out'] - df['u_out_lag3']).fillna(0).astype('float32')
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    #df['u_out_diff4'] = (df['u_out'] - df['u_out_lag4']).fillna(0).astype('float32')
    #df['cross']= df['u_in']*df['u_out']
    #df['cross2']= df['time_step']*df['u_out']


    #https://www.kaggle.com/alexxanderlarko/lgbm-sel-feat-1
    df['u_in_1st_derivative'] = (df['u_in'].diff().fillna(0) / df['time_step'].diff().fillna(0)).fillna(0)
    df['expand_mean_1sr_der'] = df.groupby('breath_id')['u_in_1st_derivative'].expanding(2).mean().reset_index(level=0,drop=True)
    #df['u_in_1st_der_mean10'] = df.groupby('breath_id')['u_in_1st_derivative'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    #df['time_diff'] = df['time_step']-df['time_step'].shift(1)
    #df['power'] = df['time_diff']*df['u_in']
    df['power']=df['time_delta']*df['u_in']
    df['power_cumsum'] = df.groupby(['breath_id'])['power'].cumsum()
    df['ewm_u_in_mean'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
    df['ewm_u_in_std'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
    df['ewm_u_in_corr'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
    df['delta_u_in'] = abs(df.groupby(df['breath_id'])['u_in'].diff().fillna(0)).reset_index(level=0,drop=True)
    #cudf
     
    df['u_in_1st_der_mean10'] = df.groupby('breath_id')['u_in_1st_derivative'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    
    #df['delta_u_in_exp'] = df.groupby(df['breath_id'])['delta_u_in'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['delta_rolling_10_mean'] = df.groupby('breath_id')['delta_u_in'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['delta_rolling_10_max'] = df.groupby('breath_id')['delta_u_in'].rolling(window=10, min_periods=1).max().reset_index(level=0,drop=True)
    #cudf end
    df['work']=((df['u_in'] + df['u_in'].shift(1).fillna(0))/2 * df['time_step'].diff().fillna(0)).clip(0,)
    df['work_roll_10']=df.groupby(df['breath_id'])['work'].rolling(window=10, min_periods=1).sum().reset_index(level=0,drop=True)
    df['work_roll_15']=df.groupby(df['breath_id'])['work'].rolling(window=15, min_periods=1).sum().reset_index(level=0,drop=True)
    
    
    df['u_in_rol_q0.1'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.1).reset_index(level=0,drop=True)
    df['u_in_rol_q0.25'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.25).reset_index(level=0,drop=True)
    df['u_in_rol_q0.5'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.5).reset_index(level=0,drop=True)
    df['u_in_rol_q0.75'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.75).reset_index(level=0,drop=True)
    df['u_in_rol_q0.9'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.9).reset_index(level=0,drop=True)     
    
    #df['u_in_diff1_back'] = df['u_in'] - df['u_in_lag_back1']
    #df['u_in_diff2_back'] = df['u_in'] - df['u_in_lag_back2']
    #df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    #df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']

    df = df.fillna(0)
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)

    return df


train_df = add_feature(train_df)
#test_df = add_feature(test_df)

ids = train_df[['id']].values.reshape(-1, 80)
#print(train_df.shape)

#print(len(X_all))
#ids.shape


In [ ]:
embedding_df.head()

In [ ]:
# def add_feature(df):
#     df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
#     df['delta'] = df['time_delta'] * df['u_in']
#     df['area'] = df.groupby('breath_id')['delta'].cumsum()

#     df = df.drop(['time_delta','delta'], axis=1)

#     df['cross']= df['u_in']*df['u_out']
#     df['cross2']= df['time_step']*df['u_out']
    
#     df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
#     df['one'] = 1
#     df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
#     df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
#     df = df.drop(['count','one'], axis=1)

#     df['breath_id_lag1']=df['breath_id'].shift(1).fillna(0)
#     df['breath_id_lag1same']=np.select([df['breath_id_lag1']==df['breath_id']], [1], 0)
#     df['u_in_lag_1'] = df['u_in'].shift(1).fillna(0) * df[f'breath_id_lag1same']
#     df['u_in_time1'] = df['u_in'] - df['u_in_lag_1']
#     df['u_out_lag_1'] = df['u_out'].shift(1).fillna(0) * df['breath_id_lag1same']

#     df['breath_id_lag2']=df['breath_id'].shift(1).fillna(0)
#     df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']], [1], 0)
#     df['u_in_lag_2'] = df['u_in'].shift(1).fillna(0) * df[f'breath_id_lag1same']
#     df['u_in_time2'] = df['u_in'] - df['u_in_lag_2']
#     df['u_out_lag_2'] = df['u_out'].shift(1).fillna(0) * df['breath_id_lag2same']

#     df['breath_id_lag3']=df['breath_id'].shift(1).fillna(0)
#     df['breath_id_lag3same']=np.select([df['breath_id_lag3']==df['breath_id']], [1], 0)
#     df['u_in_lag_3'] = df['u_in'].shift(1).fillna(0) * df[f'breath_id_lag3same']
#     df['u_in_time3'] = df['u_in'] - df['u_in_lag_3']
#     df['u_out_lag_3'] = df['u_out'].shift(1).fillna(0) * df['breath_id_lag3same']

#     df['breath_id_lag4']=df['breath_id'].shift(1).fillna(0)
#     df['breath_id_lag4same']=np.select([df['breath_id_lag4']==df['breath_id']], [1], 0)
#     df['u_in_lag_4'] = df['u_in'].shift(1).fillna(0) * df[f'breath_id_lag4same']
#     df['u_in_time4'] = df['u_in'] - df['u_in_lag_4']
#     df['u_out_lag_4'] = df['u_out'].shift(1).fillna(0) * df['breath_id_lag4same']

#     drop_columns = ['breath_id_lag1', 'breath_id_lag2', 'breath_id_lag3', 'breath_id_lag4', 
#                     'breath_id_lag1same', 'breath_id_lag2same', 'breath_id_lag3same', 'breath_id_lag4same']

#     df = df.drop(drop_columns, axis=1)

#     c_dic = {10: 0, 20: 1, 50:2}
#     r_dic = {5: 0, 20: 1, 50:2}
#     rc_sum_dic = {v: i for i, v in enumerate([15, 25, 30, 40, 55, 60, 70, 100])}
#     rc_dot_dic = {v: i for i, v in enumerate([50, 100, 200, 250, 400, 500, 2500, 1000])}    

#     df['C_cate'] = df['C'].map(c_dic)
#     df['R_cate'] = df['R'].map(r_dic)
#     df['RC_sum'] = (df['R'] + df['C']).map(rc_sum_dic)
#     df['RC_dot'] = (df['R'] * df['C']).map(rc_dot_dic)
#     df = df.drop(['R','C'], axis=1)

#     # fill na by zero
#     df = df.fillna(0)

#     return df


# train_df = add_feature(train_df)
# test_df = add_feature(test_df)




In [ ]:
#targets = train_df[['pressure']].to_numpy().reshape(-1, 80)
#u_outs = train_df[['u_out']].to_numpy().reshape(-1, 80)
#u_outs = train_df['u_out'] 
kfolds = train_df[['kfold']] #.to_numpy().reshape(-1, 80)
train_breathid = train_df.breath_id.values 
#test_breathid = test_df.breath_id.values
train_ids = train_df.id.values
#test_ids = test_df.id.values 

train_pressure = train_df.pressure.values
# test_pressure = test.pressure.values 

train_df.drop(['pressure', 'id', 'breath_id', 'kfold'], axis=1, inplace=True)
#test_df = test_df.drop(['id', 'breath_id'], axis=1)
#train_df.shape, test_df.shape

In [ ]:
!git clone https://github.com/ojus1/Time2Vec-PyTorch/ 

In [ ]:
import sys

sys.path.append('/kaggle/working/Time2Vec-PyTorch/')



In [ ]:
from Model import *
Model("sin", 8)

In [ ]:
traincols = train_df.columns

#train_df['u_out_orig']=df['u_out'].values

RS = RobustScaler()
train_df = RS.fit_transform(train_df[traincols])
train_df=train_df.astype('float32')

#train_df.shape, test_df.shape,len(traincols),len(testcols)


In [ ]:
#train_df=train_df.astype('float32')
import gc
gc.collect()
gc.collect()

In [ ]:
train_df.dtype



In [ ]:
from periodic_activations import SineActivation , CosineActivation,t2v
from Data import ToyDataset
from torch import nn
import torch

In [ ]:


class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)


def t2v1(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, tau.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape,v2.shape)
    return torch.cat([v1, v2], -1)
t2v=t2v1

class Model(nn.Module):
    def __init__(self, activation, hiddem_dim):
        super(Model, self).__init__()
        if activation == "sin":
            self.l1 = SineActivation(1, hiddem_dim)
        elif activation == "cos":
            self.l1 = CosineActivation(1, hiddem_dim)
        
        self.fc1 = nn.Linear(hiddem_dim, 2)
    
    def forward(self, x):
        #x = x.unsqueeze(1)
        x = self.l1(x)
        x = self.fc1(x)
        return x
class VPPModel(tez.Model):
    
    
    def __init__(self, learning_rate):
        hidden = [768, 512, 256, 128]
        super().__init__()
        self.learning_rate = learning_rate
        # self.loss_fct = custom_ce_loss()
        self.step_scheduler_after = "batch" #"epoch"
        self.time_step=Model("sin", 8)
        self.u_in=Model("sin", 8)
        self.r=nn.Embedding(3,2)
        self.c=nn.Embedding(3,2)

        #self.lstm = nn.LSTM(len(args.ALL_FEATURES), args.HIDDEN_SIZE, batch_first=True, bidirectional=True, dropout=0.0, num_layers=4)
        # self.head = nn.Linear(args.HIDDEN_SIZE * 2,950)
        '''
        self.head = nn.Sequential(
            nn.Linear(args.HIDDEN_SIZE * 2, args.HIDDEN_SIZE * 2),
            nn.LayerNorm(args.HIDDEN_SIZE * 2),
            nn.ReLU(),
            nn.Linear(args.HIDDEN_SIZE * 2, 950),
        )
        '''
        
        self.head = nn.Sequential(
            nn.Linear( hidden[3] * 2, hidden[3] * 2),
            nn.LayerNorm( hidden[3] * 2),
            nn.ReLU(),
            nn.Linear( hidden[3] * 2, 950),
        )
        
        self.lstm1 = nn.LSTM( len(traincols)+1+2+2, hidden[0],
                             batch_first=True, bidirectional=True)
        self.gru = nn.LSTM(2 * hidden[0], hidden[0],
                             batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(2 * hidden[0], hidden[2],
                             batch_first=True, bidirectional=True)
        self.lstm4 = nn.LSTM(2 * hidden[2], hidden[3],
                             batch_first=True, bidirectional=True)
        
        
        self._reinitialize()
        
        '''
        # LSTM
        for n, m in self.named_modules():
            if isinstance(m, nn.LSTM):
                print(f'init {m}')
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)
        '''
                        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                    #nn.init.orthogonal_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name or 'conv1' in name:
                if 'weight' in name:
                    #nn.init.xavier_uniform_(p.data)
                    nn.init.normal_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)

    
    
    
    def custom_ce_loss(self, logits=None, y=None, u_out=None):
        criterion = nn.CrossEntropyLoss()

        loss = criterion(logits.reshape(-1, 950), y.reshape(-1))

        for lag, w in [(1, 0.4), (2, 0.2), (3, 0.1), (4, 0.1)]:
            # negative lag loss
            # if target < 0, target = 0
            neg_lag_target = F.relu(y.reshape(-1) - lag)
            neg_lag_target = neg_lag_target.long()
            neg_lag_loss = criterion(logits.reshape(-1, 950), neg_lag_target)

            # positive lag loss
            # if target > 949, target = 949
            pos_lag_target = 949 - F.relu((949 - (y.reshape(-1) + lag)))
            pos_lag_target = pos_lag_target.long()
            pos_lag_loss = criterion(logits.reshape(-1, 950), pos_lag_target)

            loss += (neg_lag_loss + pos_lag_loss) * w

        return loss

    
    def monitor_metrics(self, outputs, targets, real_pressure, loss, u_out):
        w = 1 - u_out     
        #out = torch.tensor([[target_dic_inv[j.item()] for j in i] for i in torch.argmax(outputs,axis=2)]).to(args.device)
        out= target_dict_inv_array[torch.argmax(outputs,axis=2)].to(args.device) 
        mae = w * (real_pressure - out).abs()
        mae = mae.sum(-1) / w.sum(-1) 
        
        return {"ce": loss.item(), "mae":mae.mean().item()}

    def fetch_scheduler(self):
        # sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        #     self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        # )
        
        num_train_steps = int((len(train_dataset)/args.batch_size) * args.epochs)  #int(len(train_loader) * args.epochs)
        num_warmup_steps = int(num_train_steps / 10)
        #sch = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        if resume :
             print('resume')
            
             sch=  torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                  self.optimizer, T_0=40, T_mult=1, eta_min=4e-6, last_epoch=-1
              )
        else:
             sch=  torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                  self.optimizer, T_0=40, T_mult=2, eta_min=8e-6, last_epoch=-1
              )
            
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=self.learning_rate,weight_decay=3e-5)
        return opt

    def forward(self, X, y=None,real_pressure=None, u_out=None,encod_R=None,encod_C=None):
        bs = X.shape[0]
        #print(X[:,:,0].unsqueeze(-1).shape)
        x_ts=self.time_step(X[:,:,0].unsqueeze(-1) ) # we need (bs,seq len,feature )
        r_emb=self.r(encod_R)
        c_emb=self.c(encod_C)
        #print('ts',X.shape,x_ts.shape)
        X=torch.cat([x_ts,X[:,:,1:]],dim=-1)
        #print('cat',X.shape)
        X=torch.cat([X,r_emb,c_emb],dim=-1)
        #print('lstm',X.shape)
        x,_ =self.lstm1(X)
        x,_=self.gru(x)
        x, _ = self.lstm3(x)
        
        #print('dsl3',l3.size())
        x, _ = self.lstm4(x)
        
        #out, _ = self.lstm(X, None) 
        logits = self.head(x)

        if y is not None:
            loss = self.custom_ce_loss(logits, y, u_out)
            
            # loss = self.loss_fct(
            #     logits.reshape(-1, 950),
            #     y.reshape(-1)
            # ) 

            metrics = self.monitor_metrics(logits, y, real_pressure, loss, u_out)
            # print("***", metrics )
            return logits, loss, metrics

        return logits, 0, {}


In [ ]:
class VPPDataset:    
    def __init__(self, df, label_dic=None,feature_cols=traincols):
        self.dfs = [_df for _, _df in df.groupby("breath_id")]
        self.label_dic = label_dic
        self.features=feature_cols
        #self.breath_id=df.groupby("breath_id").index.values
    def __len__(self):
        return len(self.dfs)
    
    def __getitem__(self, item):
        df = self.dfs[item]
        #X = df[args.ALL_FEATURES].values     
        X = df[self.features].values  
        u_out = df['u_out'].values
        encod_r=df['encod_R'].values
        encod_c=df['encod_C'].values
        if self.label_dic is None:
            # label = None #[-1]
            # real_pressure = [-1]
            pass
        else:
            y = self.dfs[item]['pressure'].values
            #label = [self.label_dic[i] for i in y]
            label=list(map(lambda i: self.label_dic[i],y))
            real_pressure = y

        if self.label_dic is None:
            d = {
                "X": torch.tensor(X).float(),
                "u_out" : torch.tensor(u_out).long(),
            }            
        else:
            d = {
                "X": torch.tensor(X).float(),
                "y" : torch.tensor(label).long(),
                "real_pressure": torch.tensor(real_pressure).float(),
                "u_out" : torch.tensor(u_out).long(),
                "encod_R":torch.tensor(encod_r).long(),
                "encod_C":torch.tensor(encod_c).long()
            }
            #print(torch.tensor(u_out).long().sum())
        
        return d

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class VentilatorLoss(nn.Module):
    """
    Directly optimizes the competition metric
    """
    def __call__(self, logits=None, y=None, u_out=None):
        #w = 1 - u_out
        #w=torch.ones(2,80) if u_out is None else u_out
        #print(w.size(),y.size(),preds.size())
        #print(w.sum(-1),preds,'preds')
        #print(y)
        #mae = w * (y - preds.squeeze(-1)).abs()
        #mae = mae.sum(-1) / w.sum(-1)
        criterion = nn.CrossEntropyLoss()

        loss = criterion(logits.reshape(-1, 950), y.reshape(-1))

        for lag, w in [(1, 0.4), (2, 0.2), (3, 0.1), (4, 0.1)]:
            # negative lag loss
            # if target < 0, target = 0
            neg_lag_target = F.relu(y.reshape(-1) - lag)
            neg_lag_target = neg_lag_target.long()
            neg_lag_loss = criterion(logits.reshape(-1, 950), neg_lag_target)

            # positive lag loss
            # if target > 949, target = 949
            pos_lag_target = 949 - F.relu((949 - (y.reshape(-1) + lag)))
            pos_lag_target = pos_lag_target.long()
            pos_lag_loss = criterion(logits.reshape(-1, 950), pos_lag_target)

            loss += (neg_lag_loss + pos_lag_loss) * w

        return loss

        return mae

In [ ]:
class VentilatorLoss_val(nn.Module):
    """
    Directly optimizes the competition metric
    """
    def __call__(self, preds, y, u_out=None):
        #w = 1 - u_out
        out = torch.tensor([[target_dic_inv[j.item()] for j in i] for i in torch.argmax(outputs,axis=2)]).to(args.device)
         
        w=torch.ones(2,80) if u_out is None else u_out
        #print(w.size(),y.size(),preds.size())
        #print(w.sum(-1),preds,'preds')
        #print(y)
        mae = w * (y - preds.squeeze(-1)).abs()
        mae = mae.sum(-1) / w.sum(-1)

        return mae

In [ ]:
 def custom_ce_loss(self, logits=None, y=None, u_out=None):
        criterion = nn.CrossEntropyLoss()

        loss = criterion(logits.reshape(-1, 950), y.reshape(-1))

        for lag, w in [(1, 0.4), (2, 0.2), (3, 0.1), (4, 0.1)]:
            # negative lag loss
            # if target < 0, target = 0
            neg_lag_target = F.relu(y.reshape(-1) - lag)
            neg_lag_target = neg_lag_target.long()
            neg_lag_loss = criterion(logits.reshape(-1, 950), neg_lag_target)

            # positive lag loss
            # if target > 949, target = 949
            pos_lag_target = 949 - F.relu((949 - (y.reshape(-1) + lag)))
            pos_lag_target = pos_lag_target.long()
            pos_lag_loss = criterion(logits.reshape(-1, 950), pos_lag_target)

            loss += (neg_lag_loss + pos_lag_loss) * w

        return loss

In [ ]:
#criterion = VentilatorLoss()
#criterion_val= VentilatorLoss_val()
#VentilatorLoss()
def evaluate(model, loader_val):
    tb = time.time()
    was_training = model.training
    model.eval()

    loss_sum = 0
    score_sum = 0
    n_sum = 0
    y_pred_all = []
    losses_val=AverageMeter()
    for ibatch, (x, y, w) in enumerate(loader_val):
        n = y.size(0)
        x = x.to(device)
        y = y.to(device)
        w = w.to(device)

        with torch.no_grad():
            y_pred = model(x).squeeze()

        #loss = criterion(y_pred, y)
        loss = criterion_val(y_pred, y,w).mean()
        losses_val.update(loss.item(),n)
        n_sum += n
        loss_sum += n*loss.item()
        if ibatch % 10 == 0 or ibatch == (len(loader_val)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      #'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      #'Grad: {grad_norm:.4f}  '
                      'LR: {lr:.6f}  '
                      .format(
                       iepoch+1, ibatch, len(loader_val),
                       #remain=timeSince(start, float(step+1)/len(train_loader)),
                       loss=losses_val,
                       
                       lr=1e-3,
                       ))
        
        y_pred_all.append(y_pred.cpu().detach().numpy())

    loss_val = loss_sum / n_sum

    model.train(was_training)

    d = {'loss': loss_val,
         'time': time.time() - tb,
         'y_pred': np.concatenate(y_pred_all, axis=0)}

    return d


In [ ]:
from tqdm.notebook import tqdm as tqdm
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR,ReduceLROnPlateau,CosineAnnealingWarmRestarts

In [ ]:
#ids = train[['id']].values.reshape(-1, 80)

In [ ]:
'''

nfold = 5
kfold = KFold(n_splits=nfold, shuffle=True, random_state=228)
epochs = 2 if debug else 292
lr = 1e-3
batch_size = 768
max_grad_norm = 1000
import time
log = {}
oof_pred = []
oof_target = []
oof_ids=[]
for ifold, (idx_train, idx_val) in enumerate(kfold.split(X_all)):
    
    tb = time.time()
    model = Model(input_size)
    model.to(device)
    model.train()
    if ifold >=2: # due to time limit
        continue
    print('Fold %d' % ifold)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=3e-5)
    #scheduler = ReduceLROnPlateau(optimizer, factor=0.55, patience=10,verbose=True)
    scheduler = CosineAnnealingWarmRestarts(optimizer, 28, 2,eta_min=3e-5)
 
    X_train = X_all[idx_train]
    y_train = y_all[idx_train]
    w_train = w_all[idx_train]
    ids_val=ids[idx_val]
    X_val = X_all[idx_val]
    y_val = y_all[idx_val]
    w_val = w_all[idx_val]

    dataset_train = Dataset(X_train, y_train, w_train)
    dataset_val = Dataset(X_val, y_val, w_val)
    loader_train = torch.utils.data.DataLoader(dataset_train, shuffle=False,
                         batch_size=batch_size, drop_last=True)
    loader_val = torch.utils.data.DataLoader(dataset_val, shuffle=False,
                         batch_size=batch_size, drop_last=False)
    
     
    scheduler1=OneCycleLR(optimizer, 1.5e-3, total_steps=len(loader_train)*batch_size
                                 ,cycle_momentum=False, 
                                 pct_start=0.6, anneal_strategy='cos',
                                 div_factor=30.0, final_div_factor=1,
                                 last_epoch=-1, verbose=False)
     
    #scheduler = ReduceLROnPlateau(optimizer, factor=0.6, patience=10)
    
    losses_train = []
    losses_val = []
    lrs = []
    time_val = 0
    best_score = np.inf
   
    print('epoch loss_train loss_val lr time')
    best_score=np.inf
     
    for iepoch in range(epochs):
        loss_train = 0
        n_sum = 0
        losses = AverageMeter()
        for ibatch, (x, y, w) in enumerate(tqdm(loader_train)):
            n = y.size(0)
            x = x.to(device)
            y = y.to(device)
            w = w.to(device)
            optimizer.zero_grad()

            y_pred = model(x).squeeze()
            loss = criterion_val(y_pred,y )
            #loss = criterion_val(y_pred, y,w).mean()
            losses.update(loss.item(),n)
            loss_train += n*loss.item()
            n_sum += n

            loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            if ibatch % 30 == 0 or ibatch == (len(loader_train)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      #'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      #'Grad: {grad_norm:.4f}  '
                      'LR: {lr:.6f}  '
                      .format(
                       iepoch+1, ibatch, len(loader_train),
                       #remain=timeSince(start, float(step+1)/len(train_loader)),
                       loss=losses,
                       
                       #lr=1e-3,
                       lr=scheduler.get_last_lr()[-1] 
                       ))

            optimizer.step()
            scheduler.step()
        #scheduler.step()
        val = evaluate(model, loader_val)
        
        loss_val = val['loss']
        time_val += val['time']
        if val['loss']<best_score:
            best_score=val['loss']
            
            print('saving best wt',best_score)
            time.sleep(1)
            torch.save(model.state_dict(),f'fold_best{ifold}.pt')
        losses_train.append(loss_train / n_sum)
        losses_val.append(val['loss'])
        lrs.append(optimizer.param_groups[0]['lr'])

        print('%3d %9.6f %9.6f %7.3e %7.1f %6.1f' %
              (iepoch + 1,
               losses_train[-1], losses_val[-1], 
               lrs[-1], time.time() - tb, time_val))

        #scheduler.step(losses_val[-1])
        #capure oof
        #oof_pred.append( val['y_pred'])
        #oof_target.append(y_val)
        #oof_ids.append(ids_val)
     
    print('generate fold',ifold ,' oofs')
    model.load_state_dict(torch.load(f'/kaggle/working/fold_best{ifold}.pt'))
    val = evaluate(model, loader_val)
    oof_pred.append(val['y_pred'])
    oof_target.append(y_val)
    oof_ids.append(ids_val)
    #oof_target=np.concatenate(oof_target)
    #oof_ids=np.concatenate(oof_ids)
    #ofilename = 'model%d.pth' % ifold
    #torch.save(model.state_dict(), ofilename)
    #print(ofilename, 'written')

    log['fold%d' % ifold] = {
        'loss_train': np.array(losses_train),
        'loss_val': np.array(losses_val),
        'learning_rate': np.array(lrs),
        'y_pred': val['y_pred'],
        'idx': idx_val
    }
'''

In [ ]:
#train_df=train_df.astype('float32')
#train_df.dtype

In [ ]:
train_df = pd.DataFrame(train_df,columns=traincols)
#test_df = pd.DataFrame(test_df,columns=testcols)
test=False
if test:
    
    testcols = test_df.columns
    test_df = RS.transform(test_df[testcols])
    test_df=test_df.astype('float32')
    test_df['breath_id'] = test_breathid
    test_df['id'] = test_ids
    print(test_df.shape)

train_df['breath_id'] = train_breathid
#test_df['breath_id'] = test_breathid
train_df['pressure'] = train_pressure
train_df['id'] = train_ids

train_df['kfold'] = kfolds.kfold

train_df.shape

In [ ]:
train_df.head()

r_mapping={5:0,20:1,50:2}

c_mapping={10:0,20:1,50:2}

embedding_df['encod_R']=embedding_df['R'].map(r_mapping)
embedding_df['encod_C']=embedding_df['C'].map(c_mapping)

In [ ]:
#r_mapping[20]

In [ ]:
#enc_cat_mapping={'time_step':0,'encod_R':1,'encod_C':2}
#enc_cat_columns=['encod_R','encod_C','time_step']

In [ ]:
train_df=pd.concat([train_df,embedding_df.iloc[:,-2:]],axis=1)

In [ ]:
train_df.columns

In [ ]:
df_train = train_df[train_df.kfold != args.fold].reset_index(drop=True)
df_valid = train_df[train_df.kfold == args.fold].reset_index(drop=True)

train_dataset = VPPDataset(
        df = df_train[list(traincols)+['breath_id','encod_R','encod_C' ,'pressure']],
        label_dic = target_dic,
        feature_cols=traincols
    )

valid_dataset = VPPDataset(
    df = df_valid[list(traincols)+['breath_id','encod_R','encod_C' ,'pressure']],
    label_dic = target_dic,
    feature_cols=traincols
)

del train_df
gc.collect()

In [ ]:
#!cp ../input/ventilator-train-classification-ce-pytorch-optim/model_f0_r1.bin /kaggle/working/ 

#sample_data=train_dataset[0] 

In [ ]:
'''

SineAct=SineActivation
sineact = SineAct(1, 64)
cosact = CosineActivation(1, 64)
#print(torch.randn(2,80,1).shape)
print(sineact(torch.randn(2,80,1)).shape)
#print(cosact(torch.Tensor([[7]])).shape)
'''

In [ ]:
#del model
resume=False
if resume :
    model=VPPModel(learning_rate = 7e-4)
    #model.load_state_dict(torch.load('/kaggle/working/model_f0_r1.bin'))
    #model.to(args.device)
    #model.eval()
else:
    model = VPPModel(learning_rate = args.learning_rate)

es = EarlyStopping(
    monitor= "valid_mae",
    model_path=f"{args.output_folder}/model_f{args.fold}.bin",
    patience=120,
    mode="min",
    save_weights_only=True,
    delta=0.0001
)
gc.collect()

In [ ]:
'''
X=sample_data['X'].unsqueeze(0)
y=sample_data['y'].unsqueeze(0)
u_out=sample_data['u_out'].unsqueeze(0)
rp=sample_data['real_pressure'].unsqueeze(0)
eR=sample_data['encod_R'].unsqueeze(0)
eC=sample_data['encod_C'].unsqueeze(0)
model(X,y,rp,u_out,eR,eC)
'''

In [ ]:
gc.collect()

In [ ]:
print("Fold:",args.fold)

train=True
if train:

    model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=args.batch_size,
        valid_bs=3 * args.batch_size,
        device=args.device,
        epochs=180,
        callbacks=[es],
        fp16=args.fp16,
        accumulation_steps=args.accumulation_steps,
        n_jobs = 4
    ) 


In [ ]:
'''
# LSR scheduler
import torch
from torch import optim
from matplotlib import pyplot as plt
model = torch.nn.Linear(10, 2)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
steps = 10
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 40, 2)
lrs=[]
for epoch in range(300):
    for idx in range(62):
        None
        #scheduler.step()
        #print(scheduler.get_lr())
        scheduler.step()
        lrs.append(scheduler.get_lr())
'''

In [ ]:
#plt.plot(lrs)

## Validation

In [ ]:
try:
    
    del train_dataset
    gc.collect()
    gc.collect()

    from tqdm.notebook import tqdm as tqdm
    #model.load(f"{args.output_folder}/model_f{args.fold}.bin", device="cuda", weights_only=True)
    model.load(f"{args.output_folder}/model_f{args.fold}.bin", device="cuda", weights_only=True)
    valid_predictions = model.predict(valid_dataset, batch_size=args.batch_size, n_jobs=-1)

    final_valid_predictions = []
    for preds in tqdm(valid_predictions):
       #out = [[target_dic_inv[j.item()] for j in i] for i in np.argmax(preds,axis=2)]
        out= target_dict_inv_array[np.argmax(preds,axis=2)] 
        out=out.cpu().numpy()
        final_valid_predictions.extend(np.array(out).flatten().tolist())

    df_valid = df_valid[["id", "pressure", "u_out"]]
    df_valid["target"] = final_valid_predictions 

    df_valid.to_csv(f"{args.output_folder}/valid_predictions_f{args.fold}.csv", index=False)
except:
    print('x')
 

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
try:
    df_valid=pd.read_csv(f"{args.output_folder}/valid_predictions_f{args.fold}.csv")
    
    df_valid['error']=np.abs(df_valid.pressure-df_valid.target)
    print(df_valid[df_valid.u_out==-1].error.mean())
except:
    print('x')

## Test

In [ ]:
'''
del train_dataset, valid_dataset
gc.collect()

sub = pd.read_csv(f'{data_dir}/sample_submission.csv')

test_dataset = VPPDataset(
    df = test_df
)
model = VPPModel(learning_rate = args.learning_rate)

model.load(f"{args.output_folder}/model_f{args.fold}.bin", device="cuda", weights_only=True)

del train_df
gc.collect()

test_predictions = model.predict(test_dataset, batch_size=args.batch_size, n_jobs=0)
final_test_predictions = []

for preds in tqdm(test_predictions):
    out = [[target_dic_inv[j.item()] for j in i] for i in np.argmax(preds,axis=2)]
    final_test_predictions.extend(np.array(out).flatten().tolist())

sub["pressure"] = final_test_predictions
sub.to_csv(f"{args.output_folder}/test_predictions_f{args.fold}.csv", index=False)
'''

In [ ]:
! rm -rf /kaggle/working/Time2Vec-PyTorch